In [12]:
import pandas as pd
import numpy as np
import seaborn as sns
import os, glob, binascii
import matplotlib.pyplot as plt
from collections import deque
from itertools import combinations
from matplotlib.backends.backend_pdf import PdfPages
from helper_funcs import bucket_remap,process_board,ax_err_plots_player

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns',None)

def get_shape(pd,reference):
    piece = pd['piece']
    return reference[piece['shape']]

def get_color(pd,reference):
    piece = pd['piece']
    return reference[piece['color']]

# zero-indexed (row/col/cell normally 1 indexed)
def get_row(pd,reference):
    piece = pd['piece']
    return piece['y']-1+reference

# zero-indexed (row/col/cell normally 1 indexed)
def get_col(pd,reference):
    piece = pd['piece']
    return piece['x']-1+reference

# zero-indexed (row/col/cell normally 1 indexed)
def get_cell(pd,reference):
    piece = pd['piece']
    return (piece['y']-1)*6+piece['x']-1

# zero-indexed (buckets always are)
def get_bucket(pd,reference):
    buckets = list(pd['reduced_move_list'])[-reference:]
    bucket_id = {0:0,1:1,2:2,3:3,None:4}
    bucket_tuple = tuple([bucket_id[x] for x in buckets])
    return np.ravel_multi_index(bucket_tuple,tuple([5 for x in range(reference)]))

def get_quadrant(pd,reference):
    cell = get_cell(pd,None)+1
    if cell in [1,2,3,7,8,9,13,14,15]:
        return 0
    elif cell in [4,5,6,10,11,12,16,17,18]:
        return 1
    elif cell in [19,20,21,25,26,27,31,32,33]:
        return 2
    elif cell in [22,23,24,28,29,30,34,35,36]:
        return 3
    else:
        print("error calculating quadrant")
        breakpoint()

class ProcessedEnv():
    def __init__(self,args):
        self.model_features = args['MODEL_FEATURES']
        self.shape_id = {'STAR':0, 'SQUARE':1, 'TRIANGLE':2, 'CIRCLE':3}
        self.color_id = {'RED':0, 'BLUE':1, 'GREEN':2,'BLACK':2, 'YELLOW':3}
        # shape_order = ['TRIANGLE','CIRCLE','SQUARE','STAR']
        # color_order = ['RED','BLUE','YELLOW','GREEN']
        # Sanity check on feature count input
        if len(self.model_features)<1:
            print("Did you forget to add model features?")
            breakpoint()

        # Store all information about how to process each feature in following dictionary
        self.feature_info = {'shape':{'input_space':4,'func':get_shape,'reference':self.shape_id},
                       'color':{'input_space':4, 'func':get_color,'reference':self.color_id},
                       'move_row':{'input_space':6, 'func':get_row,'reference':1},
                       'move_col':{'input_space':6, 'func':get_col,'reference':1},
                       'row':{'input_space':6, 'func':get_row,'reference':0},
                       'col':{'input_space':6, 'func':get_col,'reference':0},
                       'quadrant':{'input_space':4,'func':get_quadrant,'reference':None},
                       'cell':{'input_space':36,'func':get_cell,'reference':None},
                       'bucket1':{'input_space':5,'func':get_bucket,'reference':1},
                       'bucket2':{'input_space':5**2,'func':get_bucket,'reference':2},
                       'bucket3':{'input_space':5**3,'func':get_bucket,'reference':3},
                       'bucket4':{'input_space':5**4,'func':get_bucket,'reference':4}}
        
        # Set up lists for storing past actions and board states
        self.list_memory = 5
        self.full_move_list = deque([None for x in range(self.list_memory)],maxlen=self.list_memory)
        self.reduced_move_list = deque([None for x in range(self.list_memory)],maxlen=self.list_memory)
        self.board_list = deque([None for x in range(self.list_memory)],maxlen=self.list_memory)

    def process_row(self,row):
        # Extract the x,y position of chosen piece, the chosen bucket, and current board
        x = row['x']
        y = row['y']
        bucket = row['bucket']
        board = row['proc_board']
        self.name = row['player']
        self.process_features(x,y,bucket,board)

    def process_features(self,x,y,bucket,board):
        # List of features, each entry is a dict describing the piece attributes
        self.feature_list = []
        self.reduced_move_list.append(bucket)
        self.full_move_list.append(np.nan)
        self.board_list.append(board)
        self.chosen_piece_features=None

        if len(board)>0:
            for piece in board:
                # Initialize
                feature_vals = dict.fromkeys(self.model_features)
                feature_vals['move_row']=None
                feature_vals['move_col']=None
                
                # Create standardized processing dictionary for feature processing functions
                processing_dict = {'piece':piece,'full_move_list':self.full_move_list,'reduced_move_list':self.reduced_move_list,'board_list':self.board_list}
                # Extract the 1-indexed row and column for each piece
                feature_vals['move_row'] = get_row(processing_dict,1)
                feature_vals['move_col'] = get_col(processing_dict,1)
                # Loop over all features used in the model and process the corresponding state values
                for feat in self.model_features:
                    func = self.feature_info[feat]['func']
                    ref = self.feature_info[feat]['reference']
                    feature_vals[feat]=func(processing_dict,ref)
                # Append the feature values dictionary to a list (describing all pieces on the board)
                self.feature_list.append(feature_vals)
                # If the current piece corresponds to the piece chosen by the user, store the dictionary for easy access by the models
                if feature_vals['move_col']==x and feature_vals['move_row']==y:
                    self.chosen_piece_features = feature_vals
            if self.chosen_piece_features==None:
                print("No match for chosen piece")
                print(board)
                print("x: ",x)
                print("y: ",y)
        else:
            print("BOARD ERROR")
            exit
            
    def return_feature(self,feat):
        return {'move_row':self.feature_vals['move_row'],'move_col':self.feature_vals['move_col'],'features':self.feature_vals[feat]}

    def calc_dim(self,feat_arr):
        dims = tuple(self.feature_info[feat]['input_space'] for feat in feat_arr)
        return dims

class human_bandit_model():
    def __init__(self,feats,dims,env):
        # Connect the environment to this model
        self.env = env
        # Reward scheme
        self.init_q_value = 0
        self.correct = 1
        self.incorrect = -1

        # Establish rolling memory queues for each model with configurable memory
        self.memory_horizon = 10
        self.memory=deque([],maxlen=self.memory_horizon)

        # Set up model features
        self.feats = feats
        self.feat_dims = dims
        self.in_dim, self.out_dim = np.prod(self.feat_dims), 4
        self.q_values = np.full((self.in_dim,self.out_dim),self.init_q_value,dtype=np.int8)

        # Initialize model credibility
        self.credibility_log = deque([],maxlen=self.memory_horizon)
        self.credibility=sum(self.credibility_log)
        
    def return_credibility(self):
        return self.credibility

    def return_qvals(self):
        return np.copy(self.q_values)
        
    # Process the model's memory into the current rolling q-table
    def proc_qtable(self):
        # Reset the q-table
        self.q_values = np.full((self.in_dim,self.out_dim),self.init_q_value,dtype=np.int8)

        # Iterate over memory tuples and update q-table
        for (state,action,reward) in self.memory:
            self.q_values[state,action]=reward

    # Process credibility 
    def proc_cred(self,state,action):
        # Array of q-values associated with the chosen piece
        piece_q = self.q_values[state,:]
        # Array of q-values for all pieces on the board
        board_q = self.q_values[self.board_states]
        max_board_q = board_q.max()

        # Q-value of the selected action for the chosen piece
        chosen_q = self.q_values[state,action]
        self.move_type=None

        # Credibility model
        # If the action taken is known to be correct
        if chosen_q==self.correct:
            #self.credibility_log.append(2/self.in_dim)
            self.credibility_log.append(5)
            self.move_type=4
        # If player selects an exploratory action for a piece with no known correct moves
        elif chosen_q==self.init_q_value and max_board_q==self.init_q_value:
            #self.credibility_log.append(1/self.in_dim)
            self.credibility_log.append(1)
            self.move_type=3
        # Player selects an exploratory move for a piece with known correct move
        elif chosen_q==self.init_q_value and max_board_q==self.correct:
            #self.credibility_log.append(-1/self.in_dim)
            self.credibility_log.append(1)
            self.move_type=2
        # Player selects a move known to be incorrect and had a choice
        elif chosen_q==self.incorrect and max_board_q!=self.incorrect:
            self.credibility_log.append(-7)
            #self.credibility_log.append(-3/self.in_dim)
            self.move_type=1
        # Player selects a move known to be incorrect and had no choice
        elif chosen_q==self.incorrect and max_board_q==self.incorrect:
            #self.credibility_log.append(0)
            #self.credibility_log.append(-3/self.in_dim)
            self.move_type=0
        else:
            print("ERROR: credibility processing failed to catch case")
        
        # # Debugging
        # if "72deb095616e6a5207f9" in self.env.name and self.feats[0]=='color':
        #     pass

        # Update credibility
        self.credibility=sum(self.credibility_log)
    # Process the current row of the human's experience
    def proc_row(self):
        #print(self.feats)
        #print(self.env.chosen_piece_features)
        # Information about the chosen piece (may deprecate)
        chosen_states = tuple(self.env.chosen_piece_features[feat] for feat in self.feats)
        chosen_state = np.ravel_multi_index(chosen_states,self.feat_dims)

        # Information about all the pieces on the board
        self.board_states=[]
        for item in self.env.feature_list:
            states = tuple(item[feat] for feat in self.feats)
            state = np.ravel_multi_index(states,self.feat_dims)
            self.board_states.append(state)
        # Debugging
        if "72deb095616e6a5207f9" in self.env.name and self.feats[0]=='color':
            pass
        return chosen_state
        
    def update(self,state,action,reward):
        # Process credibility (score based on the action's relation to the model's q-table)
        self.proc_cred(state,action)
        # Add tuple to memory
        self.memory.append((state,action,reward))
        # Process the model's q-table based on the current rolling memory
        self.proc_qtable()

class hl_model():
   
    def __init__(self,exp_dir,rules):
        # Gather data associated with experiment
        self.data_import(exp_dir,rules)
        
        # Model configuration
        # List of features to be used for the underlying models
        self.model_features = ['shape','color','row','col','quadrant','cell','bucket1','bucket2']
        self.args={}
        self.args['MODEL_FEATURES'] = self.model_features
        # Types of feature combinations to be used (1->unary, 2->binary combinations, etc.)
        self.combination_types = [1]
        self.env = ProcessedEnv(self.args)

    # Import relevant player files
    def data_import(self,exp_dir,tested_rules):
        # Initialize list for concatenating player records
        df_list =[]
        # Create a list containing all csv's in the given directory
        csv_list = glob.glob("**/*.csv",root_dir=exp_dir, recursive=True)
        #print(csv_list)
        # Loop over the files
        for item in csv_list:
            # Construct the import path and read data
            import_path = os.path.join(exp_dir,item)
            #print(import_path)
            df = pd.read_csv(import_path)
            #display(df)
            # Only work with data from a player's first encounter with the game
            series_num = int(df['seriesNo'].unique()[0])
            if series_num != 0:
                print(series_num)
                print("didn't meet series req")
                print(type(series_num))
                continue

            # Check the contained rules for multiple rules/players
            rules = df["#ruleSetName"].unique()
            player = df['playerId'].unique()
            if len(player)>1:
                print('------ERROR-----')
                print("File contains more than one player: ",import_path)
                print('----------------')
                break
            if len(rules)>1:
                print('------ERROR-----')
                print("Error reading the rules, multiple rules in file: ",import_path)
                print('----------------')
                break
            
            if player[0][0:2]!='ML' and ('testing' in player[0] or 'A_WORKER_ID' in player[0] or "Aria" in player[0] or player[0][0]!="A"):
                print("player name issues")
                continue

            if df.orderInSeries.max()<2:
                print("orderinseries issue")
                continue
            # if rules[0].split("/")[-1] not in tested_rules:
            #     continue

            # Get rid of finger slips (assumed when the player grabs a movable piece but misses putting it into a bucket)
            finger_slips = (df.code == 0)&(df.bx.isna())&(df.by.isna())
            df = df[~finger_slips]
            # Reset index immediately
            df.reset_index(drop=True,inplace=True)
            # Set the move number to be the corrected index
            df['move']=df.index
            # Simplify output code to errors
            df['err']=df.apply(lambda row: 0 if row.code==0 else 1, axis=1)
            df['cumulative_err']=df.err.cumsum()

            # Add processed columns
            df['bucket']=df.copy().apply(lambda x: bucket_remap(x['by'],x['bx']),axis=1)
            df['proc_board']=df.copy().apply(lambda x: process_board(x['board']),axis=1)
            #df[['shape','color','shape_ind','color_ind','id','cell','cell_ind']]=df.copy().apply(lambda x: get_attributes(x['proc_board'],x['y'],x['x']),axis=1,result_type='expand')
            # df[['shape0','shape1','shape2','shape3','color0','color1','color2','color3',
            #     'c1','c2','c3','c4','c5','c6','c7','c8','c9','c10','c11','c12',
            #     'c13','c14','c15','c16','c17','c18','c19','c20','c21','c22','c23','c24',
            #     'c25','c26','c27','c28','c29','c30','c31','c32','c33','c34','c35','c36']] = df.copy().apply(lambda x:calc_availability(x['proc_board'],shape_order,color_order),axis=1,result_type='expand')

            # Column cleanup
            df.rename(columns = {'orderInSeries':'episode','playerId':'player','#ruleSetName':'rule'},inplace=True)
            if 'precedingRules' in df:
                df.drop(columns=['seriesNo','precedingRules','timestamp','episodeId',
                                'experimentPlan','trialListId','board','p0','by','bx',
                                'moveNo'],axis=1,inplace=True)
            df_list.append(df)

            # Set for debugging
            self.debug_df = df.copy()

        # Concatenate into a single dataframe
        self.main_df = pd.concat(df_list,ignore_index=True)

    def init_models(self):
        # Initialize a list of all feature combinations to be used and populate iteratively using the combinations tool
        # Result will be a list of tuples of feature strings
        self.feature_combinations = []
        for r in self.combination_types:
            self.feature_combinations.extend(combinations(self.model_features,r))
        
        # Construct a list of models, each of which is constructed with a feature tuple and a dimension tuple
        self.models=[human_bandit_model(feat_arr,self.env.calc_dim(feat_arr),self.env) for feat_arr in self.feature_combinations]

    # Debugging utility for presenting models and associated q values
    def present_models(self):
        for model in self.models:
            print(model.feats,model.feat_dims)
            print(model.q_values)

    def player_train(self,player):
        # Train on a player's experience
        df = self.main_df.query("player==@player").copy()
        
        cred_df_list = []
        move_type_list=[]
        move_type_model_list=[]
        # Initialize models for this player
        self.init_models()

        # Loop over the rows of this player's experience
        for index,row in df.iterrows():
            # PREDICTION STAGE
            # Establish list of credibilities
            credibilities = [model.return_credibility() for model in self.models]
            # Find the best model (via the index)
            best = np.argmax(credibilities)
            best_model = self.models[best]
            
            # PROCESSING STAGE
            # Process row through the environment module
            self.env.process_row(row)
            # Action is determined
            action=row.bucket
            # Logging credibilities
            creds = {}
            move_types={}
            move_types_model={}
            # Update each model
            for model in self.models:
                # Get the raveled state for this move (prepared during env.process_row)
                state=model.proc_row()
                # Get the reward value
                reward=model.incorrect if row.err else model.correct
                # Update the model credibility and q-table
                model.update(state,action,reward)
                # Log the model credibility
                creds[model.feats]=model.return_credibility()

            cred_df_step = pd.DataFrame([creds])
            cred_df_list.append(cred_df_step)
         
            # Gather move types
            credibilities = [(model,model.return_credibility()) for model in self.models]
            sorted_cred = sorted(credibilities, key=lambda x: x[1],reverse=True)

            for i,mod in enumerate(sorted_cred[0:2]):
                model_rank = i+1
                move_types[model_rank]=mod[0].move_type
            move_types[-1]=sorted_cred[-1][0].move_type
            move_types[-2]=sorted_cred[-2][0].move_type
            move_types_df_step = pd.DataFrame([move_types])
            for i, mod_tup in enumerate(credibilities):
                move_types_model[mod_tup[0].feats]=mod_tup[0].move_type
            move_types_df_step = pd.DataFrame([move_types])
            move_types_model_df_step = pd.DataFrame([move_types_model])
            move_type_list.append(move_types_df_step)
            move_type_model_list.append(move_types_model_df_step)

        cred_df = pd.concat(cred_df_list,ignore_index=True)
        cred_df['move']=cred_df.copy().index+1
        move_type_df = pd.concat(move_type_list,ignore_index=True)
        move_type_df['move']=move_type_df.copy().index+1
        move_type_model_df = pd.concat(move_type_model_list,ignore_index=True)
        move_type_model_df['move']=move_type_model_df.copy().index+1
        #display(cred_df)
        for model in self.models:
            pass
            #print(model.feats,model.return_credibility())

        self.move_type_df = move_type_df
        return cred_df,move_type_df,move_type_model_df
            
    def experiment_train(self):
        # Establish player list
        players = self.main_df.player.unique()
        #print(players)
        log_dir = "/Users/eric/repos/gohr/bandit/human_learning/plots"
        complete_store=PdfPages(os.path.join(log_dir,"synthetic_complete.pdf"))
        # Loop over the players
        for player in players:
            #print(player)
            #print("Experience for player {}".format(player))
            creds,move_types,move_types_model = self.player_train(player)
            melted_cred = pd.melt(creds,['move'])
            melted_moves = pd.melt(move_types,['move'])
            melted_moves_model = pd.melt(move_types_model,['move'])
            melted_moves_model['model']= melted_moves_model.apply(lambda row: row.variable[0],axis=1)
            
            player_df = self.main_df.query("player==@player").copy()
            melted_cred['model'] = melted_cred.apply(lambda row: row.variable[0],axis=1)
            rule = player_df.rule.unique()[0].split('/')[-1]
            fig = plt.figure(layout='constrained', figsize=(25,14))
            subfigs = fig.subfigures(1, 2)
            ax = subfigs[0].subplots(2,1)
            #sns.lineplot(ax=ax,data=player_df,x='move',y='cumulative_err')
            ax_err_plots_player(ax[0],player_df)
            sns.lineplot(ax=ax[1],data=melted_cred.query("model in ['shape','color','row','col','quadrant','cell','bucket1','bucket2']"),x='move',y='value',hue='model',drawstyle='steps-post')
            ax[1].set_xlabel("Move")
            ax[1].set_ylabel("Credibility")

            
            custom_palette = {0:'grey',1:'red',2:'orange',3:'limegreen',4:'forestgreen'}
            
            ranking = False
            if ranking:
                ax = subfigs[1].subplots(4,1)
                for j in [1,2]:
                    ind = j-1
                    sns.scatterplot(ax=ax[ind],data=melted_moves.query("variable==@j"),x='move',y='value',hue='value',palette=custom_palette,legend=False)
                    ax[ind].set_title("Model Rank {}".format(j))
                    ax[ind].set_ylim([-0.5,4.5])
                    ax[ind].set_yticks([1,2,3,4])
                    ax[ind].set_xlabel("Move")
                    ax[ind].set_ylabel("Move Type")
                sns.scatterplot(ax=ax[2],data=melted_moves.query("variable==-2"),x='move',y='value',hue='value',palette=custom_palette,legend=False)
                sns.scatterplot(ax=ax[3],data=melted_moves.query("variable==-1"),x='move',y='value',hue='value',palette=custom_palette,legend=False)
                ax[2].set_title("Model Rank: -2")
                ax[3].set_title("Model Rank: -1")
                ax[2].set_ylim([-0.5,4.5])
                ax[2].set_yticks([1,2,3,4])
                ax[2].set_xlabel("Move")
                ax[2].set_ylabel("Move Type")
                ax[3].set_ylim([-0.5,4.5])
                ax[3].set_yticks([1,2,3,4])
                ax[3].set_xlabel("Move")
                ax[3].set_ylabel("Move Type")
            else:
                models = ['shape','color','row','quadrant']
                ax=subfigs[1].subplots(len(models),1)
                for i,mod in enumerate(models):
                    sns.scatterplot(ax=ax[i],data=melted_moves_model.query("model==@mod"),x='move',y='value',hue='value',palette=custom_palette,legend=False)
                    ax[i].set_title("Feature {}".format(mod))
                    ax[i].set_ylim([-0.5,4.5])
                    ax[i].set_yticks([1,2,3,4])
                    ax[i].set_xlabel("Move")
                    ax[i].set_ylabel("Move Type")
            # Title, close, and store
            fig.suptitle("Player: "+str(player)+", Rule: "+str(rule)+"\n")
            plt.close(fig)
            complete_store.savefig(fig,bbox_inches='tight',facecolor='w')
        self.melted_moves = melted_moves
        self.melted_cred = melted_cred
        self.move_types = move_types
        self.move_types_model = move_types_model
        self.cred_df = creds
        # Close pdfpages
        complete_store.close()

In [15]:
#exp_path = "/Users/eric/data_analysis/ambiguity4/ep/1_1_color_3m_cua"
exp_path = "/Users/eric/repos/gohr/bandit/outputs/data_generator"
rules = ["1_1_color_4m","1_2_color_4m","1_1_color_3m_cua","1_1_shape_4m","1_2_shape_4m","1_1_shape_3m_cua","quadrantNearby","quadrantNearbyTwoFree"]
model = hl_model(exp_path,rules)
model.main_df.head()

,rule,player,episode,seriesNo,acting_state,acting_q,reward,code,x,y,bx,by,board,move,err,cumulative_err,bucket,proc_board
0,1_1_shape_4m,ML_bucket1_8638c021eb2c4877a908,0,0,4,[[0 0 0 0]\n [0 0 0 0]\n [0 0 0 0]\n [0 0 0 0]...,-1,-1,3,1,0,7,"{'id': 0, 'value': [{'id': 0, 'color': 'RED', ...",0,1,1,0,"[{'id': 0, 'color': 'RED', 'shape': 'TRIANGLE'..."
1,1_1_shape_4m,ML_bucket1_8638c021eb2c4877a908,0,0,4,[[ 0 0 0 0]\n [ 0 0 0 0]\n [ 0 0 0 0]...,-1,-1,1,5,0,0,"{'id': 0, 'value': [{'id': 0, 'color': 'RED', ...",1,1,2,3,"[{'id': 0, 'color': 'RED', 'shape': 'TRIANGLE'..."
2,1_1_shape_4m,ML_bucket1_8638c021eb2c4877a908,0,0,4,[[ 0 0 0 0]\n [ 0 0 0 0]\n [ 0 0 0 0]...,-1,-1,3,4,7,7,"{'id': 0, 'value': [{'id': 0, 'color': 'RED', ...",2,1,3,1,"[{'id': 0, 'color': 'RED', 'shape': 'TRIANGLE'..."
3,1_1_shape_4m,ML_bucket1_8638c021eb2c4877a908,0,0,4,[[ 0 0 0 0]\n [ 0 0 0 0]\n [ 0 0 0 0]...,-1,-1,2,1,7,0,"{'id': 0, 'value': [{'id': 0, 'color': 'RED', ...",3,1,4,2,"[{'id': 0, 'color': 'RED', 'shape': 'TRIANGLE'..."
4,1_1_shape_4m,ML_bucket1_8638c021eb2c4877a908,0,0,4,[[ 0 0 0 0]\n [ 0 0 0 0]\n [ 0 0 0 0]...,1,0,3,1,7,7,"{'id': 0, 'value': [{'id': 0, 'color': 'RED', ...",4,0,4,1,"[{'id': 0, 'color': 'RED', 'shape': 'TRIANGLE'..."


In [16]:
model.experiment_train()
#display(model.main_df)

In [11]:
test = pd.melt(model.move_types_model,['move'])
test.head()

,move,variable,value
0,1,"(shape,)",3
1,2,"(shape,)",3
2,3,"(shape,)",3
3,4,"(shape,)",3
4,5,"(shape,)",1


In [10]:
model.move_types_model.head()

,"(shape,)","(color,)","(row,)","(col,)","(quadrant,)","(cell,)","(bucket1,)","(bucket2,)",move
0,3,3,3,3,3,3,3,3,1
1,3,3,3,3,3,3,3,3,2
2,3,3,1,3,1,3,1,3,3
3,3,3,3,3,4,3,4,4,4
4,1,1,3,3,3,3,1,1,5


In [6]:
model.cred_df.head()

,"(shape,)","(color,)","(row,)","(col,)","(quadrant,)","(cell,)","(bucket1,)","(bucket2,)",move
0,1,1,1,1,1,1,1,1,1
1,2,2,2,2,2,2,2,2,2
2,3,3,0,3,0,3,0,3,3
3,4,4,1,4,2,4,2,5,4
4,2,2,2,5,3,5,0,3,5


In [64]:
model.melted_cred.head()

,move,variable,value
0,1,"(shape,)",1
1,2,"(shape,)",2
2,3,"(shape,)",3
3,4,"(shape,)",4
4,5,"(shape,)",2


In [37]:
model.melted_moves.head()

,move,variable,value
0,1,1,3
1,2,1,3
2,3,1,3
3,4,1,3
4,5,1,4
